In [12]:
import pandas as pd
import numpy as np
file_path = 'C:/Users/poung/OneDrive/바탕 화면/capstone/dataset.csv'
df = pd.read_csv(file_path)
df[0:]

,식품코드,식품명,탄수화물(g),단백질(g),지방(g),에너지(kcal),분류
0,D219-711000000-0287,아이스크림_프릳츠 커피 아이스크림,26.96,4.35,11.30,227,간식류
1,D303-148000000-0001,라면_간편조리세트_똠양꿍요괴라면,13.49,2.41,2.77,88,밥류
2,D101-004160000-0001,국밥_돼지머리,15.94,6.70,5.16,137,밥류
3,D101-004310000-0001,국밥_순대국밥,10.38,3.17,2.28,75,밥류
4,D101-004500000-0001,국밥_콩나물,10.93,1.45,0.24,52,밥류
...,...,...,...,...,...,...,...
88195,P124-200020000-0382,기타가공품_헬씨나오P 밸런스 파우더,87.80,1.02,0.09,356,가공식품/기타
88196,P124-200020000-0383,기타가공품_현대옥 된장 시래기,9.35,8.50,2.97,89,가공식품/기타
88197,P124-200020000-0384,기타가공품_휘티 부스터 데일리 석류,26.13,1.15,0.04,109,가공식품/기타
88198,P124-200020000-0385,기타가공품_휘티 부스터 데일리 스포츠,25.72,2.91,0.31,117,가공식품/기타


In [87]:
class Person:
    
    def cal_nut(self):
        #연령별 칼로리
        if self.age <= 18:
            self.Kcal = 2700
        elif self.age <= 29:
            self.Kcal = 2600
        elif self.age <= 49:
            self.Kcal = 2400
        elif self.age <= 64:
            self.Kcal = 2200
        elif self.age <= 74:
            self.Kcal = 2300
        else:
            self.Kcal = 2000

        if self.gender == 'female':
            self.Kcal -= 400
    def __init__(self, age, gender):
        self.age = age
        self.gender = gender
        self.Kcal = 0
        self.protein = 55
        self.carbo = 324
        self.fats = 54
        self.cal_nut()

    def daily_reset(self):
        #날짜 초기화
        self.cal_nut()
        self.carbo = 324
        self.fats = 54
        self.protein = 55
    def extra_nut(self):
        #여유 에너지
        return self.Kcal,self.protein,self.carbo,self.fats
    def eat(self,car,pro,fat,kal):
        self.Kcal -= kal
        self.protein -= pro
        self.carbo -= car
        self.fats -= fat

def find_food(food_name, df):
    #음식명과 음식행 번호 반환
    food_list = df[df['식품명'].str.contains(food_name, case=False)]
    
    if food_list.empty:
        print("해당하는 음식이 없습니다.")
    else:
        print("해당하는 음식 리스트:")
        for index, row in food_list.iterrows():
            print(row['식품명'])
        return food_list.index.tolist()

def select_food(food_name, df):
    #음식 정보 반환
    food_info = df[df['식품명'].str.contains(food_name, case=False)]
    
    if food_info.empty:
        print("해당하는 음식이 없습니다.")
    else:
        print("해당하는 음식 정보:")
        return(food_info.iloc[:, 1:])
def select_food_category(category, df):
    # 분류 정보로 음식 검색
    food_info = df[df['분류'].str.contains(category, case=False)]
    
    if food_info.empty:
        print("해당하는 음식이 없습니다.")
    else:
        print("해당하는 음식 정보:")
        return food_info.iloc[:, 1:]


def deny_food(food_num):
    recommend = recommend_food(food_num, df)
    print(recommend)
def confirm_food(food_num, df):
    food_info = df.iloc[food_num, 2:6]
    if any(getattr(person, nutrient) <= food_info.iloc[i] for i, nutrient in enumerate(['carbo', 'protein', 'fats', 'Kcal'])):
        print("일일 영양섭취 초과입니다")
        deny_food(food_num)
    else:
        person.eat(*food_info)

def recommend_food(food_num, df):
    selected_food = df.iloc[food_num]
    category_name = selected_food['분류']
    target_kcal, target_protein, target_carbo, target_fats = person.extra_nut()
    
    sim_foods = df[(df['분류'] == category_name) &
                   (df['에너지(kcal)'] <= target_kcal) &
                   (df['단백질(g)'] <= target_protein) &
                   (df['탄수화물(g)'] <= target_carbo) &
                   (df['지방(g)'] <= target_fats)].copy()

    sim_foods['similarity'] = sim_foods['식품명'].apply(lambda x: similar(x, selected_food['식품명']))
    sim_foods = sim_foods.sort_values(by='similarity', ascending=False)

    return sim_foods.iloc[0:10, 1:]


def similar(a, b):
    if a == b:
        return 1.0
    else:
        n, m = len(a), len(b)
        if n > m:
            a, b = b, a
            n, m = m, n

        current_row = range(n+1)
        for i in range(1, m+1):
            previous_row, current_row = current_row, [i] + [0] * n
            for j in range(1, n+1):
                add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
                if a[j-1] != b[i-1]:
                    change += 1
                current_row[j] = min(add, delete, change)

        return 1 - (current_row[n] / max(m, n))

In [88]:
person = Person(age=25, gender='female')
print(person.extra_nut())
find_food("순대국밥",df)
confirm_food(3,df)
print(person.extra_nut())
person.daily_reset()
print(person.extra_nut())
select_food_category("밥류",df)
deny_food(3)

(2200, 55, 324, 54)
해당하는 음식 리스트:
국밥_순대국밥
양념육_더마앗 순대국밥 고기
식육간편조리_가마솥순대국밥 우골육수
밥류_한우물순대국밥
밥류_행복한한상토종순대국밥
밥류_병천식 얼큰순대국밥
밥류_순대국밥
(2125, 51.83, 313.62, 51.72)
(2200, 55, 324, 54)
해당하는 음식 정보:
              식품명  탄수화물(g)  단백질(g)  지방(g)  에너지(kcal)  분류  similarity
3         국밥_순대국밥    10.38    3.17   2.28         75  밥류    1.000000
74144     밥류_순대국밥     3.63    2.14   1.25         34  밥류    0.714286
73606  밥류_한우물순대국밥    26.19    5.71   5.24        174  밥류    0.500000
78606    국/탕류_순대국     7.40    4.01   3.24         75  밥류    0.500000
78605    국/탕류_순대국     1.00    7.50   5.83         85  밥류    0.500000
78265   국/탕류_돼지국밥     0.00    3.40   1.00         23  밥류    0.444444
83464       순대_순대    32.28    3.18   3.96        178  밥류    0.428571
171           순대국    14.95    2.35   1.75         85  밥류    0.428571
93         국수_막국수    23.90    5.80   1.61        133  밥류    0.428571
11020      국수_막국수    10.98    2.00   0.58         57  밥류    0.428571
              식품명  탄수화물(g)  단백질(g)  지방(g)  에너지(kcal)  분

In [89]:
import os
os.getcwd()

'C:\\Users\\poung'